# <font color='darkblue'> Data Modeling using Apache Cassandra </font>

In [1]:
# import needed packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath): 
    file_path_list = glob.glob(os.path.join(root,'*'))

### Note: 

The bellow code will create a unified csv file called <font color = 'navy'> event_datafile_new.csv </font> which will store all event records in one place.

<br>


In [3]:
# appending all event_data files into one csv file that will be used later in cassandra tables

full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        
        # filtering artist col, if it's empty then skip it
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [3]:
# checking total number of rows reamined in the careted csv file after applying filters

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [4]:
# creating a cluster 
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

In [5]:
# creating a keyspace
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

except Exception as e:
    print(e)

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### <font color = green> First Requirment: </font>

#### 1.1 Creating Session_songs table

In [8]:
create_table = "CREATE TABLE IF NOT EXISTS session_songs (sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(create_table)
except Exception as e:
    print(e)

#### 1.2 Inserting records to Session_songs table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO session_songs (sessionId, iteminsession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"     
        
        session.execute(query,(int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Note: 

- The bellow query will display <font color ='navy'> **Artist Name, Song Title and Length for session 338 and number of items in that session are 4.** </font>

- Sesssion Id & item in sessions are both used as compostie key that uniquely identifies each record.
<br>


In [10]:
query1 = "select artist, song_title, song_length from session_songs where sessionid=338 AND iteminsession=4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### <font color = green> Second Requirment: </font>

#### 2.1 Creating User_records table

In [11]:
create_table2 = "CREATE TABLE IF NOT EXISTS user_records (sessionId int, userId int, itemInSession int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY((sessionId, userId), itemInSession))"

try: 
    session.execute(create_table2)
except Exception as e:
    print(e)

#### 2.2 Inserting records to User_records table

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_records (sessionId, userId, itemInSession, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query,(int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

### Note: 

- The bellow query will display <font color ='navy'> **Artists Names, Songs Titles, Full Names for user Id 10 and session Id is 182.** </font>

- Sesssion Id & user Id are both used as a compostie key.
- Items in Sessions is used as a clustering key to wrok as a filtering column.
<br>


In [13]:
query2 = "SELECT artist, song_title, first_name, last_name FROM user_records WHERE userId=10 and sessionId=182"

try:
    rows= session.execute(query2)
except Exception as e:
    print(e)
for row in rows:
    print (row)


Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### <font color = green> Third Requirment: </font>

#### 3.1 Creating Song_listener table

In [7]:
create_table3 = "CREATE TABLE IF NOT EXISTS song_listener (song_title text, userId int, first_name text, last_name text, PRIMARY KEY(song_title, userId))"

try:
    session.execute(create_table3)
except Exception as e:
    print(e)

#### 3.2 Insering records to Song_listener table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_listener (song_title, userId, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        session.execute(query,(line[9], int(line[10]), line[1], line[4]))

### Note: 

- The bellow query will display <font color ='navy'> **Users' Full Names who listned to All Hands Against His Own song.** </font>

- Song title & user Id are both used as compostie key.
<br>


In [10]:
query3 = "SELECT first_name, last_name from song_listener where song_title='All Hands Against His Own'"
try:
    rows= session.execute(query3)
except Exception as e:
    print(e)
for row in rows:
    print (row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### <font color = Red> Deleting Tables: </font>

In [17]:
delete_table = "DROP TABLE session_songs"
try:
    session.execute(delete_table)
except Exception as e:
    print(e)


In [18]:
delete_table2 = "DROP TABLE user_records"
try:
    session.execute(delete_table2)
except Exception as e:
    print(e)

In [19]:
delete_table3 = "DROP TABLE song_listener"
try:
    session.execute(delete_table3)
except Exception as e:
    print(e)

### <font color = red> Closing Connection: </font>

In [20]:
session.shutdown()
cluster.shutdown()